In [60]:
import k3d
import k3d.platonic as platonic
import math
import numpy as np

plot = k3d.plot(camera_auto_fit=True)

class PointXYZ():
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
    def __sub__(self, o):
        return np.array([self.x-o.x, self.y-o.y, self.z-o.z])
    def __str__(self):
        return str(self.x) + ',' + str(self.y) + ',' + str(self.z)
    def norm(self):
        return math.sqrt(self.x ** 2 + self.y ** 2 + self.z ** 2)
    def __mul__(self, o):
        return PointXYZ(self.x * o,  self.y * o, self.z * o)


class PoseXYZ():
    def __init__(self, x, y, z, R):
        self.t = PointXYZ(x, y, z)
        self.R = R
        self.T = np.zeros((4,4))
        self.T[0:3,0:3] = R
        self.T[0, 3] = x
        self.T[1, 3] = y
        self.T[2, 3] = z
        self.T[3, 3] = 1
    def x_axis(self):
        x_end = self.R.dot(np.array([0.2, 0., 0.]))
        return ([self.t.x, x_end[0] + self.t.x], [self.t.y, x_end[1] + self.t.y], [self.t.z, x_end[2] + self.t.z])
    def y_axis(self):
        y_end = self.R.dot(np.array([0, 0.2, 0]))
        return ([self.t.x, y_end[0] + self.t.x], [self.t.y, y_end[1] + self.t.y], [self.t.z, y_end[2] + self.t.z])
    def z_axis(self):
        z_end = self.R.dot(np.array([0, 0, 0.2]))
        return ([self.t.x, z_end[0] + self.t.x], [self.t.y, z_end[1] + self.t.y], [self.t.z, z_end[2] + self.t.z])

    
class RobotArm():
    def __init__(self, k3d_plot, theta1, r1, theta2, r2):
        self.k3d_plot = k3d_plot
        self.r1 = r1
        self.theta1 = theta1
        self.r2 = r2
        self.theta2 = theta2
        
    def get_poses(self):
        c1 = math.cos(self.theta1)
        s1 = math.sin(self.theta1)
        c2 = math.cos(self.theta2)
        s2 = math.sin(self.theta2)
        r1 = self.r1
        r2 = self.r2
        R1 = np.array([[c1, -s1, 0],
                   [s1, c1, 0],
                   [0, 0, 1]])
        self.joint1_pose = PoseXYZ(0, 0, 0, R1)
        R2 = np.array([[c1, -s1*c2, s1*s2],
                       [s1, c1*c2, -c1*s2],
                       [0, s2, c2]])
        self.joint2_pose = PoseXYZ(r1 * c1, r1 * s1, 0, R2)
        self.end_effector_xyz = PointXYZ(-s1*r2*c2+r1*c1, c1*r2*c2+r1*s1, r2*s2)
        
    def update_joints(self, theta1, r1, theta2, r2):
        self.r1 = r1
        self.theta1 = theta1
        self.r2 = r2
        self.theta2 = theta2
        self.get_poses()
        self.joint1.transform.custom_matrix = self.joint1_pose.T
        self.joint2.transform.custom_matrix = self.joint2_pose.T
        self.arm1.vertices = np.array([[self.joint1_pose.t.x, self.joint1_pose.t.y, self.joint1_pose.t.z],
                      [self.joint2_pose.t.x, self.joint2_pose.t.y, self.joint2_pose.t.z]], dtype=np.float32)
        self.arm2.vertices = np.array([[self.joint2_pose.t.x, self.joint2_pose.t.y, self.joint2_pose.t.z],
                      [self.end_effector_xyz.x, self.end_effector_xyz.y, self.end_effector_xyz.z]], dtype=np.float32)
        
    def init_plot(self):
        self.get_poses()
        joint_color = 0xff0000
        self.joint1 = platonic.Cube(size=0.1).mesh
        self.joint2 = platonic.Cube(size=0.1).mesh
        self.joint1.color = joint_color
        self.joint2.color = joint_color
        self.joint1.transform.custom_matrix = self.joint1_pose.T
        self.joint2.transform.custom_matrix = self.joint2_pose.T
        r1 = np.array([[self.joint1_pose.t.x, self.joint1_pose.t.y, self.joint1_pose.t.z],
                      [self.joint2_pose.t.x, self.joint2_pose.t.y, self.joint2_pose.t.z]], dtype=np.float32)
        self.arm1 = k3d.line(
            r1, 
            shader='mesh', 
            width=0.05)
        r2 = np.array([[self.joint2_pose.t.x, self.joint2_pose.t.y, self.joint2_pose.t.z],
                      [self.end_effector_xyz.x, self.end_effector_xyz.y, self.end_effector_xyz.z]], dtype=np.float32)
        self.arm2 = k3d.line(
            r2, 
            shader='mesh', 
            width=0.05)
        self.k3d_plot += self.joint1
        self.k3d_plot += self.joint2
        self.k3d_plot += self.arm1
        self.k3d_plot += self.arm2
        
def to_radian(deg):
    return deg / 180.0 * math.pi

def get_joints_poses(*, theta1, r1, theta2, r2):
    c1 = math.cos(theta1)
    s1 = math.sin(theta1)
    c2 = math.cos(theta2)
    s2 = math.sin(theta2)
    R1 = np.array([[c1, -s1, 0],
                   [s1, c1, 0],
                   [0, 0, 1]])
    joint1_pose = PoseXYZ(0, 0, 0, R1)
    R2 = np.array([[c1, -s1*c2, s1*s2],
                   [s1, c1*c2, -c1*s2],
                   [0, s2, c2]])
    joint2_pose = PoseXYZ(r1 * c1, r1 * s1, 0, R2)
    end_effector_xyz = PointXYZ(-s1*r2*c2+r1*c1, c1*r2*c2+r1*s1, r2*s2)
    return (joint1_pose, joint2_pose, end_effector_xyz)


def get_jacobian(*, theta1, r1, theta2, r2, dof=4):
    c1 = math.cos(theta1)
    s1 = math.sin(theta1)
    c2 = math.cos(theta2)
    s2 = math.sin(theta2)
    if dof == 2:
    # with respect to theta1, theta2
        return np.array([[-c1*r2*c2-r1*s1, s2*s1*r2],
                         [-s1*r2*c2+r1*c1, -s2*c1*r2],
                         [0, c2*r2]])
    if dof == 3:
    # with respect to theta1, theta2, r2
        return np.array([[-c1*r2*c2-r1*s1, s2*s1*r2, -s1*c2],
                         [-s1*r2*c2+r1*c1, -s2*c1*r2, c1*c2],
                         [0, c2*r2, s2]])
    if dof == 4:
    # with respect to theta1, r1, theta2, r2
        return np.array([[-c1*c2*r2-r1*s1, c1, s1*s2*r2, -s1*c2],
                         [-s1*c2*r2+r1*c1, s1, -s2*c1*r2, c1*c2],
                         [0, 0, c2*r2, s2]])
    if dof == 300:
        # with respect to theta1, r1, theta2
        return np.array([[-c1*c2*r2-r1*s1, c1, s1*s2*r2],
                         [-s1*c2*r2+r1*c1, s1, -s2*c1*r2],
                         [0, 0, c2*r2]])
    return -1

def clamp(num, min_value, max_value):
    return max(min(num, max_value), min_value)

def get_joints_velocity(*, theta1, r1, theta2, r2, end_effector_speed, dof=4):
    # WARNING: it may not find a solution. Gauss newton method, maybe should use LM?
    step_scale = 1 # Maybe set trust region?
    max_theta_v = to_radian(2)
    max_linear_v = 1E-1

    v = [0, 0, 0, 0]
    J = get_jacobian(theta1=theta1, r1=r1, theta2=theta2, r2=r2, dof=dof)
    INF = np.transpose(J).dot(J)
    eps = 1E-4 # Solve singularity issue and make it converge faster...
    tmp = np.linalg.inv(INF + np.eye(INF.shape[0]) * eps).dot(np.transpose(J)).dot(end_effector_speed) * step_scale
    if dof == 2:
        v[0] = clamp(tmp[0], -max_theta_v, max_theta_v) # theta1
        v[2] = clamp(tmp[1], -max_theta_v, max_theta_v) # theta2
    if dof == 3:
        v[0] = clamp(tmp[0], -max_theta_v, max_theta_v) # theta1
        v[2] = clamp(tmp[1], -max_theta_v, max_theta_v) # theta2
        v[3] = clamp(tmp[2], -max_linear_v, max_linear_v) # r2
    if dof == 300:
        v[0] = clamp(tmp[0], -max_theta_v, max_theta_v) # theta1
        v[1] = clamp(tmp[1], -max_linear_v, max_linear_v) # r1
        v[2] = clamp(tmp[2], -max_theta_v, max_theta_v) # theta2
    if dof == 4:
        v[0] = clamp(tmp[0], -max_theta_v, max_theta_v) # theta1
        v[1] = clamp(tmp[1], -max_linear_v, max_linear_v) # r1
        v[2] = clamp(tmp[2], -max_theta_v, max_theta_v) # theta2
        v[3] = clamp(tmp[3], -max_linear_v, max_linear_v) # r2
    return v

arm = RobotArm(plot, to_radian(0), 1, to_radian(0), 1)
arm.init_plot()
traj_points = k3d.points(positions=np.array([arm.end_effector_xyz.x, arm.end_effector_xyz.y, arm.end_effector_xyz.z]), 
                   point_size=0.2, shader='3d', color=0xffff00)
goal_points = k3d.points(positions=np.array([arm.end_effector_xyz.x, arm.end_effector_xyz.y, arm.end_effector_xyz.z]), 
                   point_size=0.2, shader='3d', color=0xff0000)
plot += traj_points
plot += goal_points
plot.display()

Output()

In [66]:
theta1_start = to_radian(30)
theta2_start = to_radian(60)
r1_start = 2
r2_start = 1
(joint1_pose_start, joint2_pose_start, end_effector_xyz_start) = get_joints_poses(theta1=theta1_start, r1=r1_start,
                                                             theta2=theta2_start, r2=r2_start)
theta1_curr = theta1_start
r1_curr = r1_start
theta2_curr = theta2_start
r2_curr = r2_start
end_effector_xyz_curr = end_effector_xyz_start
arm.update_joints(theta1_curr, r1_curr, theta2_curr, r2_curr)
traj_points.positions = np.array([[end_effector_xyz_start.x, end_effector_xyz_start.y, end_effector_xyz_start.z]], dtype=np.float32)

theta1_goal = to_radian(60)
theta2_goal = to_radian(30)
r1_goal = 1
r2_goal = 2
(joint1_pose_goal, joint2_pose_goal, end_effector_xyz_goal) = get_joints_poses(theta1=theta1_goal, r1=r1_goal,
                                                                              theta2=theta2_goal, r2=r2_goal)
goal_points.positions = np.array([[end_effector_xyz_goal.x, end_effector_xyz_goal.y, end_effector_xyz_goal.z]], dtype=np.float32)


In [67]:
import time
k = 0
iters = 500
termination_eps = 1E-3
dof = 4
use_geometry = True
use_geometry = False

time.sleep(1)
print('start executing')

if use_geometry:
    r = math.sqrt(end_effector_xyz_goal.x ** 2 + end_effector_xyz_goal.y ** 2)
    if dof == 3:
        assert(r1_curr < r)
        r2_curr = math.sqrt(end_effector_xyz_goal.norm() ** 2 - r1_curr ** 2)
        dof = 2
    if dof == 4:
        while r <= r1_curr:
            r1_curr /= 2.
        r2_curr = math.sqrt(end_effector_xyz_goal.norm() ** 2 - r1_curr ** 2)
        dof = 2
    (joint1_pose_curr, joint2_pose_curr, end_effector_xyz_curr) = get_joints_poses(theta1=theta1_curr, r1=r1_curr,
                                                                                   theta2=theta2_curr, r2=r2_curr)
    traj_points.positions = np.append(traj_points.positions, 
                                      [[end_effector_xyz_curr.x, end_effector_xyz_curr.y, end_effector_xyz_curr.z]], 
                                      axis=0)
    arm.update_joints(theta1_curr, r1_curr, theta2_curr, r2_curr)

while k <= iters:
    v_xyz = end_effector_xyz_goal - end_effector_xyz_curr
    v = get_joints_velocity(theta1=theta1_curr, r1=r1_curr,
                            theta2=theta2_curr, r2=r2_curr,
                            end_effector_speed=v_xyz, dof=dof)
    theta1_curr = np.unwrap(np.array([theta1_curr + v[0]]))
    r1_curr = r1_curr + v[1]
    theta2_curr = np.unwrap(np.array([theta2_curr + v[2]]))
    r2_curr = r2_curr + v[3]
    arm.update_joints(theta1_curr, r1_curr, theta2_curr, r2_curr)
    (joint1_pose_curr, joint2_pose_curr, end_effector_xyz_curr) = get_joints_poses(theta1=theta1_curr, r1=r1_curr,
                                                                                    theta2=theta2_curr, r2=r2_curr)
    traj_points.positions = np.append(traj_points.positions, 
                                      [[end_effector_xyz_curr.x, end_effector_xyz_curr.y, end_effector_xyz_curr.z]], 
                                      axis=0)
    err = np.linalg.norm(end_effector_xyz_goal - end_effector_xyz_curr)
    if err < termination_eps or k == iters:
        print(f'Finial error {err}')
        print(f'iter {k}')
        break
    k += 1
    time.sleep(0.2)

start executing
Finial error 1.698573652237332e-07
iter 18
